In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'fourier'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_fourier_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,36507.40,1.25,-2.25,-0.28,0.78,0.12,-0.01,-0.46,0.14,-0.46,...,-0.02,-0.01,-0.04,0.03,0.00,0.00,-0.00,-0.00,-0.00,-0.00
3,1.25,46636.51,-0.12,-0.02,1.40,0.66,0.14,0.58,0.33,0.18,...,-0.05,-0.00,-0.02,0.00,0.01,0.00,-0.00,0.00,-0.00,0.00
4,-2.25,-0.12,42415.32,-0.35,-0.90,0.05,0.85,-0.07,0.12,0.30,...,-0.10,0.03,0.03,-0.01,0.00,-0.01,0.00,0.00,0.00,-0.00
5,-0.28,-0.02,-0.35,21975.28,-0.68,0.21,-0.44,-0.14,-0.23,-0.32,...,0.02,0.02,0.01,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
6,0.78,1.40,-0.90,-0.68,14371.06,-0.12,-0.06,-0.19,-0.22,-0.13,...,-0.01,0.02,0.01,-0.01,0.00,-0.00,0.00,-0.00,-0.00,-0.00
7,0.12,0.66,0.05,0.21,-0.12,8408.76,-0.10,0.07,0.21,0.10,...,0.01,0.05,-0.00,0.01,-0.00,0.00,0.00,0.00,-0.00,-0.00
8,-0.01,0.14,0.85,-0.44,-0.06,-0.10,6275.84,-0.10,0.11,0.18,...,-0.01,-0.02,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00
9,-0.46,0.58,-0.07,-0.14,-0.19,0.07,-0.10,5348.56,-0.06,0.15,...,0.02,-0.00,-0.01,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00
10,0.14,0.33,0.12,-0.23,-0.22,0.21,0.11,-0.06,3770.53,-0.10,...,0.01,-0.00,0.01,-0.01,-0.00,0.00,0.00,-0.00,-0.00,0.00
11,-0.46,0.18,0.30,-0.32,-0.13,0.10,0.18,0.15,-0.10,3227.52,...,0.00,-0.01,-0.00,0.01,-0.00,0.00,-0.00,0.00,-0.00,-0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,0.00003,-0.00006,-0.00001,0.00003,0.00001,-0.00000,-0.00003,0.00001,-0.00004,...,-0.00001,-0.00000,-0.00003,0.00003,0.00001,0.00000,-0.00000,-0.00002,-0.00000,-0.00000
3,0.00003,1.00000,-0.00000,-0.00000,0.00005,0.00003,0.00001,0.00004,0.00003,0.00001,...,-0.00002,-0.00000,-0.00002,0.00000,0.00002,0.00001,-0.00000,0.00000,-0.00000,0.00001
4,-0.00006,-0.00000,1.00000,-0.00001,-0.00004,0.00000,0.00005,-0.00000,0.00001,0.00003,...,-0.00005,0.00001,0.00003,-0.00001,0.00000,-0.00003,0.00001,0.00002,0.00001,-0.00000
5,-0.00001,-0.00000,-0.00001,1.00000,-0.00004,0.00002,-0.00004,-0.00001,-0.00003,-0.00004,...,0.00002,0.00001,0.00001,0.00000,-0.00001,-0.00001,-0.00000,-0.00001,0.00003,0.00000
6,0.00003,0.00005,-0.00004,-0.00004,1.00000,-0.00001,-0.00001,-0.00002,-0.00003,-0.00002,...,-0.00001,0.00001,0.00002,-0.00002,0.00001,-0.00000,0.00001,-0.00001,-0.00001,-0.00000
7,0.00001,0.00003,0.00000,0.00002,-0.00001,1.00000,-0.00001,0.00001,0.00004,0.00002,...,0.00001,0.00004,-0.00000,0.00003,-0.00002,0.00001,0.00000,0.00001,-0.00001,-0.00000
8,-0.00000,0.00001,0.00005,-0.00004,-0.00001,-0.00001,1.00000,-0.00002,0.00002,0.00004,...,-0.00001,-0.00002,0.00000,0.00000,-0.00000,-0.00001,0.00001,-0.00001,-0.00001,-0.00001
9,-0.00003,0.00004,-0.00000,-0.00001,-0.00002,0.00001,-0.00002,1.00000,-0.00001,0.00004,...,0.00002,-0.00000,-0.00002,-0.00000,-0.00000,0.00000,-0.00000,0.00002,0.00001,-0.00000
10,0.00001,0.00003,0.00001,-0.00003,-0.00003,0.00004,0.00002,-0.00001,1.00000,-0.00003,...,0.00002,-0.00000,0.00002,-0.00003,-0.00000,0.00003,0.00001,-0.00001,-0.00001,0.00001
11,-0.00004,0.00001,0.00003,-0.00004,-0.00002,0.00002,0.00004,0.00004,-0.00003,1.00000,...,0.00001,-0.00001,-0.00001,0.00002,-0.00001,0.00001,-0.00001,0.00000,-0.00001,-0.00001


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

5.44849232469342

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.83956487e+07 5.61561864e+07 4.14008051e+07 1.50418600e+07
 6.42488553e+06 2.20218514e+06 1.22659987e+06 8.84595592e+05
 4.41563585e+05 3.20728630e+05 1.35702214e+05 1.20575560e+05
 1.05709894e+05 4.71299260e+04 2.56922914e+04 2.25801343e+04
 1.29101226e+04 7.50521353e+03 2.72893208e+03 2.29517938e+03
 1.63153410e+03 1.07258543e+03 3.14737746e+02 1.72363180e+02
 2.93507484e+01 1.64553144e+01 2.90155669e-01 1.90239478e-01
 7.93328513e-03 1.72597636e-03 1.60209819e-05 1.34886624e-23]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-5.347011e-02,9.866267e-01,-1.527669e-01,-1.552401e-02,-8.640785e-03,-4.730356e-03,-0.003489,-0.002943,-0.002064,-0.001766,...,-0.000057,-0.000106,-0.000018,-0.000014,-0.000001,-0.000002,-3.210110e-07,-1.528115e-07,-1.790078e-08,-1.719305e-09
1,-1.209783e-01,1.448715e-01,9.816052e-01,-2.356027e-02,-1.265222e-02,-6.783297e-03,-0.004947,-0.004208,-0.002939,-0.002504,...,-0.000083,-0.000150,-0.000024,-0.000020,-0.000002,-0.000003,-4.038993e-07,-1.689551e-07,-1.947636e-08,-5.922886e-09
2,9.904661e-01,6.994075e-02,1.104793e-01,-3.677690e-02,-1.783105e-02,-9.279604e-03,-0.006751,-0.005721,-0.003972,-0.003407,...,-0.000109,-0.000203,-0.000034,-0.000025,-0.000003,-0.000003,-5.940646e-07,-3.428462e-07,-3.656209e-08,-8.500688e-09
3,3.181198e-02,2.059836e-02,2.403196e-02,9.978279e-01,-4.182937e-02,-1.602208e-02,-0.011118,-0.009228,-0.006305,-0.005360,...,-0.000167,-0.000313,-0.000051,-0.000040,-0.000005,-0.000005,-9.366814e-07,-4.930812e-07,-1.792005e-08,-1.037505e-08
4,1.641539e-02,1.201537e-02,1.357612e-02,3.964375e-02,9.978768e-01,-3.345483e-02,-0.020069,-0.016038,-0.010436,-0.008749,...,-0.000267,-0.000497,-0.000081,-0.000064,-0.000007,-0.000008,-1.385902e-06,-7.756601e-07,-8.623588e-08,-2.160110e-08
5,8.337525e-03,6.380434e-03,7.083695e-03,1.529783e-02,3.005320e-02,9.961228e-01,-0.062843,-0.038948,-0.020114,-0.016108,...,-0.000438,-0.000816,-0.000135,-0.000104,-0.000013,-0.000013,-2.354104e-06,-1.158826e-06,-1.430029e-07,-3.343612e-08
6,6.051574e-03,4.682967e-03,5.149103e-03,1.063032e-02,1.831497e-02,5.529401e-02,0.989508,-0.122058,-0.033950,-0.025044,...,-0.000587,-0.001101,-0.000180,-0.000140,-0.000016,-0.000018,-3.050081e-06,-1.686592e-06,-1.870714e-07,-5.321325e-08
7,6.137344e-03,4.740474e-03,5.242697e-03,1.057293e-02,1.758760e-02,4.252593e-02,0.114965,0.988703,-0.062541,-0.041469,...,-0.000825,-0.001553,-0.000256,-0.000198,-0.000023,-0.000025,-4.516945e-06,-2.155447e-06,-2.292460e-07,-6.471776e-08
8,3.823509e-03,2.977472e-03,3.285063e-03,6.491085e-03,1.033123e-02,2.036321e-02,0.033754,0.049649,0.987048,-0.136681,...,-0.001046,-0.001968,-0.000320,-0.000253,-0.000029,-0.000030,-5.593785e-06,-2.987684e-06,-3.512260e-07,-6.680819e-08
9,4.012318e-03,3.115866e-03,3.427668e-03,6.752457e-03,1.060844e-02,2.002999e-02,0.030819,0.041457,0.127986,0.986101,...,-0.001511,-0.002845,-0.000465,-0.000360,-0.000042,-0.000044,-8.303538e-06,-4.174203e-06,-4.758454e-07,-1.178685e-07


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.053470114841844,
 0.8551284596793657,
 0.8895207406633439,
 0.002172077612199841,
 0.0021231620353496794,
 0.0038771836244086977,
 0.010491740688751028,
 0.011297208707517603,
 0.012952027380024544,
 0.01389909644010423,
 1.2772875761518003,
 1.0245490338327221,
 1.0489196256664357,
 1.0769520480817696,
 1.0329425952739748,
 1.0695253804560247,
 1.0351328294902238,
 1.0707533099185143,
 1.0835359282960986,
 0.989638315683394,
 0.9908285419905494,
 1.0622153252493423,
 0.025931828838852033,
 0.9421841015174006,
 0.03780973072516358,
 0.03787625048297083,
 1.5862801809299274,
 0.4166415982238175,
 0.02605060542690707,
 0.026203339301627215,
 0.030540537420808866,
 0.02907490083930142]